In [1]:
import pandas as pd
import geopandas as gp
from src.utils.map_vis import geojson_per_row
import folium

# load localidades

In [2]:
mex_localidades = gp.read_file('data/localidades_mexico_16.geojson')

# load municipalities covering localidades

In [3]:
from src.utils import fix_spanish_encoding

In [4]:
# http://www.inegi.org.mx/geo/contenidos/geoestadistica/m_g_0.aspx
shp_urban = gp.read_file('data/mexico/mglu2014v6_2/mglu2015v6_2.shp')
shp_state = gp.read_file('data/mexico/mge2014v6_2/mge2015v6_2.shp')
shp_munic = gp.read_file('data/mexico/mgm2014v6_2/mgm2015v6_2.shp')

# get Urban id
shp_urban.NOM_LOC = shp_urban.NOM_LOC.apply(fix_spanish_encoding)
shp_urban['URBAN_ID'] = shp_urban.CVE_ENT + shp_urban.CVE_MUN + shp_urban.CVE_LOC

# get the full data frame
shp_all = shp_urban.merge(shp_state.drop('geometry', axis=1)).merge(shp_munic.drop('geometry', axis=1))

In [5]:
munic_localidades_pairs = gp.sjoin(shp_munic, mex_localidades)['index_right'].reset_index()

In [6]:
def get_inx_area(row):
    m = shp_munic.loc[row['index']].geometry
    c = mex_localidades.loc[row['index_right']].geometry
    return m.intersection(c).area

In [7]:
munic_localidades_pairs['inx_area'] = munic_localidades_pairs.apply(get_inx_area, axis=1)

In [8]:
mindices = munic_localidades_pairs[munic_localidades_pairs.inx_area>1e-4]

In [9]:
munic_intersecting_cities = shp_munic.loc[mindices['index']]

# load metro areas with the names of the localidades

In [10]:
from shapely.ops import cascaded_union

In [11]:
metro = gp.read_file('data/mexico/Mapa_de_delimitacion_de_zonas_metropolitanas_2010/ZM_2010.shp')
sun_ids = [31, 21, 32, 28, 24, 11, 12, 41, 37, 3, 17,2,25,45]
sun_names = ['Monterrey, Nuevo León', 'Guadalajara, Jalisco', 'Oaxaca de Juárez, Oaxaca', 'Cuernavaca, Morelos', 'Toluca de Lerdo, México', 'Juárez, Chihuahua', 'Chihuahua, Chihuahua', 'Villahermosa, Tabasco', 'Cancún, Quintana Roo', 'Mexicali, Baja California', 'Acapulco de Juárez, Guerrero', 'Tijuana, Baja California', 'Morelia, Michoacán de Ocampo', 'Nuevo Laredo, Tamaulipas','Culiacán Rosales, Sinaloa', 'Valle de Mexico, Distrito Federal']

In [12]:
suns = []
for cve,name in zip(sun_ids,sun_names):
    sun = metro[metro.CVE_SUN==cve]
    geometry = cascaded_union(sun.geometry.tolist())
    pob = sun.POB_2010.sum()
    suns.append({'name': name, 'cve_sun': cve, 'population': pob, 'nom_sun':sun.NOM_SUN.unique()[0], 'geometry':geometry})
#     break

sun = metro[(metro.CVE_SUN==13) & (metro.CVE_ENT=='09')]
geometry = cascaded_union(sun.geometry.tolist())
pob = sun.POB_2010.sum()
suns.append({'name': sun_names[-1], 'cve_sun': 13, 'population': pob, 'nom_sun':sun.NOM_SUN.unique()[0], 'geometry':geometry})

suns=gp.GeoDataFrame(suns)

suns.crs = metro.crs

suns = suns.to_crs(epsg=4326)

In [13]:
# metro[(metro.CVE_SUN==13) & (metro.CVE_ENT=='09')]

In [15]:
some_map = folium.Map(location=[24.062199, -102.870324], zoom_start=5)
geojson_per_row(suns, name='metropolitan-yellow', color='yellow', tip_cols=['name','population'], some_map=some_map)
geojson_per_row(munic_intersecting_cities, name='munics-red', color='red', tip_cols=['NOM_MUN','CVE_ENT','CVE_MUN',], some_map=some_map)
geojson_per_row(mex_localidades, name='cities-blue', tip_cols=['URBAN_ID', 'NOM_ENT', 'NOM_MUN','NOM_LOC',], some_map=some_map)

folium.LayerControl().add_to(some_map)
some_map.save('tmp/mexico locali and munics and metro.html')